# Import libs

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

# Data

In [2]:
# Load the MNIST dataset
# Using the arg 'as_supervised=True' will load the dataset already split as training set and test set
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info=True)

# Training and test dataset
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


## Find train set cardinality 

In [3]:
num_train_samples = mnist_info.splits['train'].num_examples
# Force the result to be integer
num_train_samples = tf.cast(num_train_samples, tf.int64)
num_train_samples

<tf.Tensor: id=284, shape=(), dtype=int64, numpy=60000>

## Find validation set cardinality 

In [4]:
# Let's create the validation set from the training set
# Let's be the 10% of the traing set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# Force the result to be integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_validation_samples

<tf.Tensor: id=286, shape=(), dtype=int64, numpy=6000>

## Find test set cardinality

In [5]:
num_test_samples = mnist_info.splits['test'].num_examples
# Force the result to be integer
num_test_samples = tf.cast(num_test_samples, tf.int64)
num_test_samples

<tf.Tensor: id=288, shape=(), dtype=int64, numpy=10000>

## Scale data and make it numerical

In [6]:
# Define the scaling fn
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. # the final . is to keep the result a float
    return image, label

In [7]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [8]:
test_data = mnist_test.map(scale)

## Shuffle the data

In [9]:
 BUFFER_SIZE = 10000

In [10]:
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

## Validation data

In [11]:
# Now that we have a shuffled, scaled training data, let's take the correct number of samples to make the validation set
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
validation_data

<TakeDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

## Train data

In [12]:
# Train data and validation data don't have to overlap
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
train_data

<SkipDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

## Batch the data sets

In [13]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE) # Split data in batches

# NB: only one batch cause we are not back-propagating
# Needs to be batch as format
validation_data = validation_data.batch(num_validation_samples) 
test_data = test_data.batch(num_test_samples)

## Make the validation data compatible in format to train and test data
The MNIST data is iterable and in 2-tuple format (as_supervided=True)

In [14]:
validation_input, validation_targets = next(iter(validation_data))

# Model

## Outline the model

In [15]:
input_size = 784
output_size = 10
hidden_layer_size = 500

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

## Choose the optimizer and loss function

In [16]:
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training

In [17]:
NUM_EPOCH = 5

model.fit(
    train_data,
    epochs=NUM_EPOCH, 
    validation_steps=1,
    validation_data=(validation_input, validation_targets), 
    verbose=2
)

Epoch 1/5


540/540 - 11s - loss: 0.2149 - accuracy: 0.9351 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 10s - loss: 0.0858 - accuracy: 0.9739 - val_loss: 0.0804 - val_accuracy: 0.9770
Epoch 3/5
540/540 - 11s - loss: 0.0628 - accuracy: 0.9805 - val_loss: 0.0680 - val_accuracy: 0.9785
Epoch 4/5
540/540 - 12s - loss: 0.0451 - accuracy: 0.9856 - val_loss: 0.0495 - val_accuracy: 0.9870
Epoch 5/5
540/540 - 11s - loss: 0.0379 - accuracy: 0.9876 - val_loss: 0.0525 - val_accuracy: 0.9850


## Test the model

In [18]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0920 - accuracy: 0.9755


In [19]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.55%
